In [2]:
import pandas as pd
import numpy as np
import urllib.request as urllib2
from bs4 import BeautifulSoup

In [3]:
# note: FFCalculator goes back to 2018 for half ppr, 2010 for ppr, and earlier for standard
ppr = 1.0
cur_season = 2019

In [15]:
def _scrape_adp(ppr, cur_season):
    df = None
    
    if ppr==0.:
        ppr_str = '/standard'
    elif ppr==0.5:
        ppr_str = '/half-ppr'
    elif ppr==1.0:
        ppr_str = '/ppr'
    
    for season in range(2010, cur_season):
        url = 'https://fantasyfootballcalculator.com/adp{0}/12-team/all/{1}'.format(ppr_str, str(season))
        page = urllib2.urlopen(urllib2.Request(url, headers={'User-Agent' : "Magic Browser"}))
        soup = BeautifulSoup(page, 'html.parser')

        data = []
        for tr in soup.find_all('tr'):
            ths = tr.find_all('th')
            if len(ths)>0 and len(header)==0:
                for header_val in ths:
                    header.append(header_val.text.strip())
            
            tds = tr.find_all('td')
            if len(tds)>0:
                # all rows with player data start with a number and player name, e.g. "1. Melvin Gordon"
                # use that fact to check if this table row has player data in it
                row = []
                for row_val in tds:
                    row.append(row_val.text.strip())
                data.append(row)

        temp_df = pd.DataFrame(data, columns=header)
        temp_df['Season'] = season
        
        if df is None:
            df = temp_df.copy(deep=True)
        else:
            df = df.append(temp_df, ignore_index=True)
                
    return df

In [23]:
df = _scrape_adp(ppr, cur_season)

df.head()

,#,Pick,Name,Pos,Team,Overall,Std.Dev,High,Low,TimesDrafted,,Season
0,1,1.01,Chris Johnson,RB,TEN,1.4,1.3,1.01,2.12,422,,2010
1,2,1.02,Adrian Peterson,RB,MIN,2.3,1.3,1.01,2.11,342,,2010
2,3,1.03,Ray Rice,RB,BAL,3.2,1.5,1.01,2.10,203,,2010
3,4,1.05,Maurice Jones-Drew,RB,JAX,4.7,1.5,1.01,2.09,325,,2010
4,5,1.05,Andre Johnson,WR,HOU,4.8,2.6,1.01,5.01,427,,2010


In [25]:
df.columns = ['ADPRank','Pick','Player','Pos','Team','Overall','Std.Dev','High','Low','TimesDrafted','','Season']
df = df[['Season','Player','Pos','ADPRank']]

In [26]:
df.to_csv('data/seasonADPs.csv', index=False)